In [1]:
import findspark
import os
findspark.init(spark_home='/opt/spark')
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-arm64/'
os.environ['PYSPARK_PYTHON'] = '/usr/local/bin/python3.11'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/datasaku/repo/datasaku/.venv/bin/python3'

In [2]:
!echo $SPARK_HOME

/opt/spark


In [3]:
!ls $SPARK_HOME/jars

HikariCP-2.5.1.jar
JLargeArrays-1.5.jar
JTransforms-3.1.jar
RoaringBitmap-0.9.45.jar
ST4-4.0.4.jar
activation-1.1.1.jar
aircompressor-0.27.jar
algebra_2.12-2.0.1.jar
annotations-17.0.0.jar
antlr-runtime-3.5.2.jar
antlr4-4.9.3.jar
antlr4-runtime-4.9.3.jar
antlr4-runtime-4.9.3.jar.1
aopalliance-repackaged-2.6.1.jar
arpack-3.0.3.jar
arpack_combined_all-0.1.jar
arrow-format-12.0.1.jar
arrow-memory-core-12.0.1.jar
arrow-memory-netty-12.0.1.jar
arrow-vector-12.0.1.jar
audience-annotations-0.5.0.jar
avro-1.11.2.jar
avro-ipc-1.11.2.jar
avro-mapred-1.11.2.jar
aws-java-sdk-bundle-1.12.262.jar
blas-3.0.3.jar
bonecp-0.8.0.RELEASE.jar
breeze-macros_2.12-2.1.0.jar
breeze_2.12-2.1.0.jar
cats-kernel_2.12-2.1.1.jar
chill-java-0.10.0.jar
chill_2.12-0.10.0.jar
commons-cli-1.5.0.jar
commons-codec-1.16.1.jar
commons-collections-3.2.2.jar
commons-collections4-4.4.jar
commons-compiler-3.1.9.jar
commons-compress-1.23.0.jar
commons-crypto-1.1.0.jar
commons-dbcp-1.4.jar
commons-io-2.16.1.jar
commons-lang-2.6.ja

In [4]:
!ls /opt/spark/jars

HikariCP-2.5.1.jar
JLargeArrays-1.5.jar
JTransforms-3.1.jar
RoaringBitmap-0.9.45.jar
ST4-4.0.4.jar
activation-1.1.1.jar
aircompressor-0.27.jar
algebra_2.12-2.0.1.jar
annotations-17.0.0.jar
antlr-runtime-3.5.2.jar
antlr4-4.9.3.jar
antlr4-runtime-4.9.3.jar
antlr4-runtime-4.9.3.jar.1
aopalliance-repackaged-2.6.1.jar
arpack-3.0.3.jar
arpack_combined_all-0.1.jar
arrow-format-12.0.1.jar
arrow-memory-core-12.0.1.jar
arrow-memory-netty-12.0.1.jar
arrow-vector-12.0.1.jar
audience-annotations-0.5.0.jar
avro-1.11.2.jar
avro-ipc-1.11.2.jar
avro-mapred-1.11.2.jar
aws-java-sdk-bundle-1.12.262.jar
blas-3.0.3.jar
bonecp-0.8.0.RELEASE.jar
breeze-macros_2.12-2.1.0.jar
breeze_2.12-2.1.0.jar
cats-kernel_2.12-2.1.1.jar
chill-java-0.10.0.jar
chill_2.12-0.10.0.jar
commons-cli-1.5.0.jar
commons-codec-1.16.1.jar
commons-collections-3.2.2.jar
commons-collections4-4.4.jar
commons-compiler-3.1.9.jar
commons-compress-1.23.0.jar
commons-crypto-1.1.0.jar
commons-dbcp-1.4.jar
commons-io-2.16.1.jar
commons-lang-2.6.ja

In [5]:
!java -version

openjdk version "17.0.13" 2024-10-15
OpenJDK Runtime Environment (build 17.0.13+11-Debian-1deb11u1)
OpenJDK 64-Bit Server VM (build 17.0.13+11-Debian-1deb11u1, mixed mode, sharing)


In [6]:
import pyspark
from pyspark.sql import SparkSession
import os
import socket

conf = pyspark.SparkConf().setAll([
    ('spark.driver.host', socket.gethostbyname(socket.gethostname()))
    , ('spark.app.name', 'test')
    , ('spark.master', "spark://spark-master.spark-dev.svc.cluster.local:7077")
    # , ("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-spark_2.12:3.2.1,io.unitycatalog:unitycatalog-spark_2.12:0.2.0")
    , ("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    , ('spark.hadoop.fs.s3a.endpoint', "http://minio-service.minio-dev.svc.cluster.local:6544")
    # for minio spark
    , ('spark.hadoop.fs.s3a.access.key','minio')
    , ('spark.hadoop.fs.s3a.secret.key', 'minio123')
    , ('spark.hadoop.fs.s3a.path.style.access', 'true')
    , ("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # for iceberg minio jdbc
    , ("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    , ('spark.sql.catalog.spark_catalog', 'io.unitycatalog.spark.UCSingleCatalog')
    , ('spark.sql.catalog.unity', 'io.unitycatalog.spark.UCSingleCatalog')
    , ("spark.sql.catalog.unity.catalog-impl", "io.delta.sql.DeltaSparkSessionExtension")
    , ('spark.sql.catalog.unity.uri', 'http://unity-service.unity-dev.svc.cluster.local:4566')
    , ('spark.sql.catalog.unity.token', '')
    , ('spark.sql.defaultCatalog', 'unity')
    # , ('spark.sql.catalog.nessie_catalog.warehouse', 's3a://unity/')
    ])
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/18 09:42:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark.sql("SHOW SCHEMAS in unity").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [8]:
spark.sql("SHOW TABLES IN unity.default").show()  

+---------+-----------------+-----------+
|namespace|        tableName|isTemporary|
+---------+-----------------+-----------+
|  default|        marksheet|      false|
|  default|marksheet_uniform|      false|
|  default|          numbers|      false|
|  default|   user_countries|      false|
+---------+-----------------+-----------+



In [9]:
spark.sql("SELECT * FROM unity.default.numbers").show()

AnalysisException: [DELTA_TABLE_NOT_FOUND] Delta table `default`.`numbers` doesn't exist.

In [16]:
spark.sql("SELECT * FROM unity.information_schema.tables")

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `unity`.`information_schema`.`tables` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [unity, information_schema, tables], [], false


In [9]:
spark.sql('CREATE catalog unity_ext;')

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'catalog'.(line 1, pos 7)

== SQL ==
CREATE catalog unity_ext;
-------^^^


In [10]:
spark.sql('CREATE SCHEMA demo;')

DataFrame[]

In [11]:
spark.sql("SHOW SCHEMAS").show()

+---------+
|namespace|
+---------+
|  default|
|     demo|
+---------+



In [32]:
spark.sql("""
CREATE TABLE
unity.demo.mytable11 (id INT, desc STRING) 
USING delta 
""")

AnalysisException: [SCHEMA_NOT_FOUND] The schema `demo` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a catalog, verify the current_schema() output, or qualify the name with the correct catalog.
To tolerate the error on drop use DROP SCHEMA IF EXISTS.

In [29]:
spark.sql("desc unity.demo.mytable9")

AnalysisException: [DELTA_PATH_DOES_NOT_EXIST] file:/opt/unitycatalog/s3a:/unity/demo/mytable9 doesn't exist, or is not a Delta table.;
DescribeRelation false, [col_name#324, data_type#325, comment#326]
+- ResolvedTable io.unitycatalog.spark.UCSingleCatalog@cafcb56, demo.mytable9, DeltaTableV2(org.apache.spark.sql.SparkSession@56b09494,file:/opt/unitycatalog/s3a:/unity/demo/mytable9,Some(CatalogTable(
Catalog: unity
Database: demo
Table: mytable9
Created Time: Sat Jan 18 10:05:28 UTC 2025
Last Access: UNKNOWN
Created By: Spark 
Type: EXTERNAL
Provider: DELTA
Location: file:///opt/unitycatalog/s3a:/unity/demo/mytable9)),Some(demo.mytable9),None,Map())


In [19]:
spark.sql("SHOW TABLES IN demo").show()  

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|     demo| mytable6|      false|
|     demo| mytable7|      false|
|     demo| mytable8|      false|
+---------+---------+-----------+



In [14]:
spark.sql(
"""
INSERT INTO unity.demo.mytable6 VALUES (1, 'test');
"""
)

AnalysisException: [DELTA_TABLE_NOT_FOUND] Delta table `demo`.`mytable6` doesn't exist.;
AppendData RelationV2[] unity.demo.mytable6 unity.demo.mytable6, false
+- LocalRelation [col1#128, col2#129]


In [31]:
spark.sql("SHOW SCHEMAS").show()

+---------+
|namespace|
+---------+
|     demo|
+---------+



In [30]:
spark.sql('select current_schema()').show()

+------------------+
|current_database()|
+------------------+
|              demo|
+------------------+

